In [5]:
import pandas as pd
import numpy as np
import copy
import random
from math import sqrt
import scipy.stats as sts

from tqdm import tqdm_notebook

from supplement_package import game
from supplement_package import variables_pecan

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()

In [3]:
from supplement_package.game.stackelberg import StackelbergPlayer

In [4]:
import gurobipy as gp

from supplement_package.gurobi_implementation.gurobi import GurobiSolution

## Data preliminaries 

In [5]:
agent_keys = [661, 1642, 2335, 2361, 2818, 3039, 3456, 3538, 4031, 4373, 4767, 5746, 6139, 7536, 7719, 7800, 7901, 7951, 8156, 8386, 8565, 9019, 9160, 9922, 9278]

dataframe_dict = dict()
for key in agent_keys:
    dataframe_dict.update({key : pd.read_csv('/Users/ishilov/Documents/risk_paper/risk_paper/data/df_{}.csv'.format(key))})

community_size = len(dataframe_dict)

for key in agent_keys:
    cond_min = (dataframe_dict[key]['demand'].quantile(0.001) <= dataframe_dict[key]['demand'])
    cond_max = (dataframe_dict[key]['demand'] <= dataframe_dict[key]['demand'].quantile(0.999))
    dataframe_dict[key] = dataframe_dict[key][cond_min & cond_max]

In [6]:
def generate_new_params(sample_size, community_size):
    res = {}

    for sample in range(sample_size):
        A_tilde = [random.uniform(0,1) for i in range(community_size)]
        B_tilde = [random.uniform(0,1) for i in range(community_size)]

        a = [random.uniform(0,1) for i in range(community_size)]
        b = [random.uniform(0,1) for i in range(community_size)]
        d = [random.uniform(0,1) for i in range(community_size)]

        #d_target = [[random.uniform(0,8) for j in range(len(probabilities))] for i in range(community_size)]
        #g_res = [[random.uniform(0,3) for j in range(len(probabilities))] for i in range(community_size)]

        #g_res = np.array(g_res)
        #d_target = np.array(d_target)

        risk_aversion = [random.uniform(0,1) for i in range(community_size)]

        res.update({sample : {'A_tilde' : A_tilde,
                        'B_tilde' : B_tilde,
                        'a' : a,
                        'b' : b,
                        'd' : d,
                        'risk_aversion' : risk_aversion}})

    res_reformed = {(i, key) : res[i][key] for i in range(sample_size) for key in res[0].keys()}
    mindx = pd.MultiIndex.from_tuples(res_reformed.keys())
    df = pd.DataFrame(list(res_reformed.values()), index = mindx)
    df.to_csv(f'../data/param_{sample_size}.csv')


In [7]:
def text_to_adj_matrix(matrix_path):
    res = []
    with open(matrix_path) as file:
        for s in file:
            string = ''.join(s.strip().strip(',').split(', '))
            lst_temp = [int(sym) for sym in string]

            res.append(lst_temp)

    return res

In [8]:
#generate_new_params(2000)

In [9]:
def read_df_param(sample_size):
    df_param = pd.read_csv(f'../data/param_{sample_size}.csv')
    df_param.rename({'Unnamed: 0' : 'Sample', 'Unnamed: 1' : 'Parameter'}, axis=1, inplace= True)
    df_param.set_index(['Sample', 'Parameter'], inplace=True)

    return df_param

In [10]:
def distribution_build(sample_size, agent_keys):
    res = {}
    
    for key in agent_keys:
        #chunks_demand = int(demand_dict[key][0].size / sample_size)
        #chunks_generation = int(solar_dict[key][0].size / sample_size)


        #probas_demand = [np.trapz(demand_dict[key][1][i * sample_size : (i + 1) * sample_size],
        #                            demand_dict[key][0][i * sample_size : (i + 1) * sample_size])
        #                            for i in range(chunks_demand)]

        #probas_generation = [np.trapz(generation_dict[key][1][i * sample_size : (i + 1) * sample_size],
        #                            generation_dict[key][0][i * sample_size : (i + 1) * sample_size])
        #                            for i in range(chunks_generation)]

        #res_demand = plt.hist(np.random.choice(demand_dict[key][0], size = sample_size, p = probabilities), bins = int(sample_size / 2))
        #probas_update = res_demand[0] / res_demand[0].sum() if key == 661 else res[661]['probabilities']
        #res.update({key : 
        #            {'values' : res_demand[1], 
        #            'probabilities' : probas_update}})

        hist_demand = plt.hist(dataframe_dict[key]['demand'], bins = sample_size)
        probas_demand, values_demand = hist_demand[0], hist_demand[1]
        probas_demand = probas_demand / probas_demand.sum()

        if 'solar' in dataframe_dict[key].columns:
            hist_solar = plt.hist(dataframe_dict[key][dataframe_dict[key]['solar'] >= 0]['solar'], bins = sample_size)
            probas_solar , values_solar  = hist_solar[0], hist_solar[1]
            probas_solar = probas_demand / probas_demand.sum()

        res.update({key : 
                    {'probas_demand' : probas_demand,
                    'values_demand' : values_demand,
                    'probas_solar' : probas_solar,
                    'values_solar' : values_solar}})

    return res 

In [11]:
def scenario_sampling(sample_size, agent_keys, main_key = 661):
    distribution = distribution_build(sample_size, agent_keys)

    probabilities = distribution[main_key]['probas_demand']

    d_target = []
    g_res = []
    for key in agent_keys:
        d_target.append(distribution[key]['values_demand'][:-1])
        g_res.append(distribution[key]['values_solar'][:-1])

    return probabilities, d_target, g_res

In [12]:
def param_input(df, index = 0):
    A_tilde = list(df.loc[index].loc['A_tilde'])
    B_tilde = list(df.loc[index].loc['B_tilde'])
    a = list(df.loc[index].loc['a'])
    b = list(df.loc[index].loc['b'])
    d = list(df.loc[index].loc['d'])
    risk_aversion = list(df.loc[index].loc['risk_aversion'])

    for i, RA in enumerate(risk_aversion):
        if RA > 0.95:
            risk_aversion[i] = risk_aversion[i] - 0.1

    return A_tilde, B_tilde, a, b, d, risk_aversion

In [13]:
param_amount = 2000

#generate_new_params(param_amount,community_size=community_size)
df_param = read_df_param(param_amount)

In [14]:
def sample_to_csv(scenario_amount, probabilities, d_target, g_res):
    pd.DataFrame(d_target).to_csv(f'../data/df_d_target_{scenario_amount}.csv')
    pd.DataFrame(g_res).to_csv(f'../data/df_g_res_{scenario_amount}.csv')
    pd.DataFrame(probabilities).to_csv(f'../data/df_probabilities_{scenario_amount}.csv')

In [15]:
def sample_from_csv(scenario_amount):
    df_d_target = pd.read_csv(f'../data/df_d_target_{scenario_amount}.csv').drop('Unnamed: 0', axis = 1)
    df_g_res = pd.read_csv(f'../data/df_g_res_{scenario_amount}.csv').drop('Unnamed: 0', axis = 1)
    df_probabilities =pd.read_csv('../data/df_probabilities_100.csv').drop('Unnamed: 0', axis = 1)
    
    probabilities = df_probabilities.values.squeeze()


    d_target = []
    for _, row in df_d_target.iterrows():
        d_target.append(row.values)

    g_res = []
    for _, row in df_g_res.iterrows():
        g_res.append(row.values)

    return probabilities, d_target, g_res
    

In [16]:
probabilities, d_target, g_res = sample_from_csv(100)
#A_tilde, B_tilde, a, b, d, risk_aversion = param_input(df_param)

In [17]:
def agents_list_only_IC_PBP_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res):
    agents = []
    StackelbergPlayer.community_size = community_size
    StackelbergPlayer.probabilities = probabilities

    epsilon = 0.0000001
    alpha = [[proba/(1 - risk_aversion[i]) - epsilon for proba in probabilities] for i in range(community_size)]
    #alpha = [[0.2 for proba in probabilities] for i in range(community_size)]
    gamma = [100000 for proba in probabilities]

    j_max = [100 for i in range(community_size)]



    for i in range(community_size):
        agent = StackelbergPlayer(i, d_target[i], g_res[i], a[i], b[i], d[i], 
                    A_tilde[i], B_tilde[i], D_min[i], D_max[i], 
                    G_min[i], G_max[i], risk_aversion[i], Kappa[i], Cost[i], connection_matrix[i],
                    probabilities = probabilities,
                    alpha = alpha[i], 
                    gamma = gamma, 
                    insurance_bound=j_max[i])
        
        agents.append(agent)

    return agents

In [18]:
def vars_to_dict(model, list_vars):
    var_names = []
    
    for var in model.getVars():
        var_names.append(var.VarName)

    dict_res = {}
    for name, var in zip(var_names, list_vars):
        dict_res.update({name : var})

    return dict_res 


In [19]:
def gurobi_experiment(df_param, index, err_track, connection_matrix, probabilities, d_target, g_res, solution_type, verbosity = 0):

    A_tilde, B_tilde, a, b, d, risk_aversion = param_input(df_param, index = index)

    if solution_type == 'only_IC':
        agents = agents_list_only_IC_PBP_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res)

    model_1 = gp.Model()
    setup = GurobiSolution(agents=agents,
                    model = model_1,
                    solution_type=solution_type)

    model_1.setParam('OutputFlag', verbosity)

    setup.build_model()

    try:
        model_1.optimize()

        list_vars = model_1.X
        dict_vars = vars_to_dict(model_1, list_vars)
        objective_val = model_1.getObjective().getValue()     
        
    except:
        err_track.append(index)
        list_vars = ['err']
        objective_val = 'err'
        dict_vars = vars_to_dict(model_1, list_vars)

    return dict_vars, objective_val, model_1

In [20]:
D_min = [0 for i in range(community_size)]
D_max = [dataframe_dict[i].demand.max() for i in dataframe_dict.keys()]

G_min = [0 for i in range(community_size)]
G_max = [dataframe_dict[i].grid.max() for i in dataframe_dict.keys()]

Kappa = [[10 if i!=j else 0 for i in range(community_size)] for j in range(community_size)]

Cost = [[1 for i in range(community_size)] for j in range(community_size)]


In [21]:
connection_matrix_2 = text_to_adj_matrix('../matrices/matrix_2.txt')

In [22]:
def experiment(probabilities, connection_matrix, d_target, g_res,
                df_param, solution_type='only_IC', verbosity = 0):

    #if solution_type == 'only_IC':
    #    agents = agents_list_only_IC_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res)

    total_results= {}
    err_track = []
    for index in tqdm_notebook(df_param.index.levels[0]):
        vars, objective, model = gurobi_experiment(df_param, index, err_track, connection_matrix, probabilities, d_target, g_res, solution_type, verbosity)
        total_results.update({index: {'vars' : vars,
                                    'objective' : objective}})


    results_vars = pd.DataFrame(data = [list(total_results[0]['vars'].values())],
                                        columns=total_results[0]['vars'].keys())

    results_vars['objective'] = total_results[0]['objective']

    for i in tqdm_notebook(range(1, len(total_results))):
        df_temp = pd.DataFrame(data = [list(total_results[i]['vars'].values())],
                                            columns=total_results[i]['vars'].keys())
                                
        df_temp['objective'] = total_results[i]['objective']

        results_vars = pd.concat([results_vars, df_temp], ignore_index=True)

    return total_results, results_vars, model

## Experiment

In [23]:
res_only_IC, df_only_IC, model_only_IC = experiment(probabilities, connection_matrix_2, d_target, g_res, df_param, solution_type='only_IC')

/var/folders/98/gq_0lybs5k55th2hc4jstmyw00m7tv/T/ipykernel_11883/4039935650.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index in tqdm_notebook(df_param.index.levels[0]):


  0%|          | 0/2000 [00:00<?, ?it/s]

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-04-02


/var/folders/98/gq_0lybs5k55th2hc4jstmyw00m7tv/T/ipykernel_11883/4039935650.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(1, len(total_results))):


  0%|          | 0/1999 [00:00<?, ?it/s]

In [24]:
df_only_IC

,D_0_0,D_0_1,D_0_2,D_0_3,D_0_4,D_0_5,D_0_6,D_0_7,D_0_8,D_0_9,...,J_24_91,J_24_92,J_24_93,J_24_94,J_24_95,J_24_96,J_24_97,J_24_98,J_24_99,objective
0,3.184002e-02,8.738969e-02,1.428530e-01,1.982271e-01,0.253509,0.308695,0.363782,0.418765,0.473642,0.528408,...,2.755311e-07,2.498341e-07,3.003140e-07,2.531837e-07,3.193265e-07,2.270103e-07,2.940727e-07,4.777555e-07,5.232408e-07,2.381871
1,7.678474e-11,1.446691e-10,1.668090e-10,9.241889e-04,0.055895,0.110845,0.165776,0.220686,0.275575,0.330444,...,1.897241e+00,1.915672e+00,1.934622e+00,1.954131e+00,1.975449e+00,1.999482e+00,2.029870e+00,2.048493e+00,2.067221e+00,4.719951
2,2.862572e-11,5.535126e-02,1.106677e-01,1.659490e-01,0.221194,0.276404,0.331379,0.386429,0.441627,0.496850,...,1.613462e+00,1.637610e+00,1.661753e+00,1.685899e+00,1.710094e+00,1.734296e+00,1.758520e+00,1.782836e+00,1.807156e+00,-0.644434
3,4.918689e-11,5.539208e-02,1.107640e-01,1.661156e-01,0.221447,0.276757,0.332047,0.387316,0.442423,0.497755,...,8.841325e-01,9.086426e-01,9.331522e-01,9.576623e-01,9.821720e-01,1.006682e+00,1.031192e+00,1.055702e+00,1.080212e+00,6.984750
4,2.887963e-09,2.121820e-08,5.397962e-02,1.077665e-01,0.161348,0.214711,0.267838,0.320713,0.373316,0.425624,...,8.190971e-07,6.500050e-07,8.166826e-07,6.508371e-07,7.852641e-07,5.299927e-07,7.256500e-07,1.309211e-06,1.331648e-06,4.991250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1.345987e-10,2.189264e-10,1.981821e-10,1.097935e-02,0.065065,0.119076,0.173013,0.226871,0.280651,0.334349,...,1.073937e+00,1.098372e+00,1.122858e+00,1.147314e+00,1.171823e+00,1.196297e+00,1.220832e+00,1.245357e+00,1.269868e+00,8.303430
1996,5.659716e-09,1.266760e-08,2.078171e-08,6.146162e-03,0.067641,0.129131,0.190614,0.252092,0.313562,0.375027,...,1.525393e+00,1.549840e+00,1.574322e+00,1.598828e+00,1.623335e+00,1.647843e+00,1.672350e+00,1.696860e+00,1.721368e+00,4.373061
1997,3.938720e-10,6.268358e-10,5.657254e-10,1.224803e-09,0.031903,0.085626,0.139266,0.192820,0.246287,0.299663,...,1.122754e+00,1.147264e+00,1.171773e+00,1.196284e+00,1.220793e+00,1.245304e+00,1.269813e+00,1.294322e+00,1.318832e+00,9.317258
1998,8.112137e-02,1.373223e-01,1.934395e-01,2.494701e-01,0.305411,0.361258,0.417009,0.472658,0.528203,0.583638,...,1.220601e+00,1.243144e+00,1.265915e+00,1.289252e+00,1.312859e+00,1.336101e+00,1.359275e+00,1.382577e+00,1.406001e+00,5.331028


In [25]:
df_only_IC.to_csv(f'../data/only_IC_PBP_results_{len(df_only_IC)}.csv')

In [26]:
def agent_utility_from_df_row(df_row, agents):
    res = []
    for agent in agents:
        eta_idx = f'eta_{agent.id}'
        agent_obj = df_row[eta_idx]

        for proba in agent.probabilities_ind:
            u_idx = f'u_{agent.id}_{proba}'
            w_idx = f'W_{agent.id}_{proba}'
            j_idx = f'J_{agent.id}_{proba}'

            agent_obj += (agent.alpha[proba] * df_row[j_idx]
                        + agent.gamma[proba] * df_row[w_idx]
                        + agent.probabilities[proba] / (1 - agent.risk_aversion) * df_row[u_idx])

        res.append(agent_obj)

    return res

In [27]:
def IC_utility_from_df_row(df_row, agents):
    res = 0

    for agent in agents:
        for proba in agent.probabilities_ind:
            j_idx = f'J_{agent.id}_{proba}'

            res += - agent.alpha[proba] * df_row[j_idx] + agent.probabilities[proba] * df_row[j_idx]

    return res

In [28]:
agent_objectives = []
for index, row in df_only_IC.iterrows():
    A_tilde, B_tilde, a, b, d, risk_aversion = param_input(df_param, index)
    agents = agents_list_only_IC_PBP_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix_2, d_target, g_res)
    agent_objectives.append(agent_utility_from_df_row(row, agents))

In [29]:
agent_objectives

[[0.24104319688554407,
  0.3208931343624193,
  0.1882839450351193,
  0.772233390615011,
  -0.22112264849268934,
  0.3491358359044636,
  -0.7346660239586428,
  -0.028482469393438684,
  0.28772987296486885,
  0.04556731750513059,
  1.2414036429661215,
  0.47112298644254286,
  0.15648461693303628,
  0.5825653853976215,
  0.3634836090508551,
  -0.0158259938233575,
  0.37014095982680406,
  0.6305817077433055,
  0.5980206213668304,
  -0.310030565286194,
  0.6837964062022016,
  0.08027309953378117,
  -0.3392090230853985,
  0.30325505703903116,
  -3.6548068470150232],
 [0.20703334890564698,
  -0.064260154960628,
  0.3577459340787012,
  0.3941531908075438,
  0.0766093254684547,
  0.2992924930266917,
  -0.0012033326527465034,
  -0.009432287110872293,
  0.4421488211289525,
  1.2249261864867438,
  4.095104572316221,
  0.6575736371613492,
  0.8411248193246421,
  -0.3536758516564388,
  0.036010641938205225,
  -0.36665954007461987,
  1.9177246994965875,
  0.482850287123183,
  1.3331593596189404,
  -3

In [30]:
dict_df_param = df_param.to_dict()
for i in range(community_size):
    for trial_idx, trial in enumerate(agent_objectives):
        dict_df_param[f'{i}'].update({(trial_idx, 'objective') : trial[i]})

df_param = pd.DataFrame(dict_df_param).sort_index(level=0)
df_param.to_csv(f'../data/only_IC_PBP_params_{2000}_with_obj.csv')

In [31]:
IC_objective = []

for index, row in df_only_IC.iterrows():
    A_tilde, B_tilde, a, b, d, risk_aversion = param_input(df_param, index)
    agents = agents_list_only_IC_PBP_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix_2, d_target, g_res) 
    IC_objective.append(IC_utility_from_df_row(row, agents))

In [32]:
IC_objective = pd.Series(IC_objective, name='IC objective only_IC_PBP')
IC_objective.to_csv('../data/IC_objective_only_IC_PBP_2000.csv')

In [6]:
IC_objective = pd.read_csv('../data/IC_objective_only_IC_PBP_2000.csv')
IC_objective.describe()

,Unnamed: 0,IC objective only_IC_PBP
count,2000.000000,2000.000000
mean,999.500000,-1.414946
std,577.494589,0.446451
min,0.000000,-3.920896
25%,499.750000,-1.692477
50%,999.500000,-1.366736
75%,1499.250000,-1.091953
max,1999.000000,-0.310562


In [1]:
columns = [f'J_{i}_{j}' for i in range(25) for j in range(100)]

df_only_IC[columns].describe().loc['max'].describe()

NameError: name 'df_only_IC' is not defined